# Data Import, Cleaning, and Preparation

This module is used to query the postgreSQL database in order to retrieve the Yelp and Violations dataset. There will be several steps to do this:
* Import flattened violations dataset. We will use a SQL query to perform aggregation of violation data by restaurant and inspeciton date. 
* Aggregate the Yelp Data
    * Reviews for a given establishment will be aggregated so that reviews *after* the previous inspection (or the earliest review date) and *before* the date of a given inspection are in one batch. 
    * Aggregate "count" features using this same logic
    * Combine the review document for a restaurant into a CLOB using the same logic
* LEFT JOIN the violations dataset to the Yelp data after aggregationis complete for both datasets

In [13]:
import psycopg2 as psy
import pandas as pd
import re

In [14]:
#set up connection to our DB
conn = psy.connect(database="sterndsyelp", 
                        user="mvsternds", 
                        password="nyustern123!", 
                        host="sterndsyelp.cawzspvmqd5q.us-east-1.rds.amazonaws.com", 
                        port="5432"
                       )
#open cursor and check our tables in the DB
cur = conn.cursor()
cur.execute("SELECT table_name FROM information_schema.tables"
            " WHERE table_schema='public'" 
            " AND table_type='BASE TABLE'")
rows =cur.fetchall()
print(pd.DataFrame(rows))

                       0
0               business
1                checkin
2             trnt_insps
3  violations_pittsburgh
4                   tips
5                reviews


In [15]:
cur.execute("SELECT * FROM public.restaurants LIMIT 50000")
biz = pd.DataFrame(cur.fetchall())

cur.execute("SELECT * FROM public.checkin LIMIT 50000")
checkins = pd.DataFrame(cur.fetchall())

cur.execute("SELECT * FROM public.reviews LIMIT 50000")
reviews = pd.DataFrame(cur.fetchall())

**NOTE: ONLY LIMITING to 50 rows during build phase to limit processing time. **

In [16]:
biz.columns = ['bizID', 'name', 'address', 'zip', 'neighborhood', 'lat','long', 'categories','attributes','is_open','review_count','hours','stars']
biz.describe()

,bizID,name,address,zip,neighborhood,lat,long,categories,attributes,is_open,review_count,hours,stars
count,8138,8138,8138,8138,8138,8138,8138,8138,8138,8138,8138,8138,8138
unique,8138,6532,6711,3071,69,7210,7207,3997,6892,2,321,3758,9
top,tBDkO-vfXrXMnLJu2sF0Hw,Starbucks,,,,43.653226,-79.3831843,"['Coffee & Tea', 'Food']",,1,3,,3.5
freq,1,132,39,42,1340,28,28,213,167,6063,785,2520,2113


In [17]:
reviews.columns = ['type','cool','bizID','reviewID','userID','stars','text','useful','funny','date']
#get dummies for star rating column
reviews = pd.concat([reviews, pd.get_dummies(reviews['stars'], prefix='stars')], axis=1)
reviews.head()

,type,cool,bizID,reviewID,userID,stars,text,useful,funny,date,stars_1,stars_2,stars_3,stars_4,stars_5
0,b'review',0,b'-DPq2CaW-mBYZ3hfed4pxw',b'dO9EjbIxMKwTuPXtpo-K2Q',b'r8KTa0JhI3cUn2RmN-FvfA',1,b'Buyer beware! This company apparently cares ...,1,0,b'2014-04-08',1,0,0,0,0
1,b'review',0,b'2Ka-a7ayZy6fcxqdTxmyhA',b'DqFeGMtiAOih6qzEhJfn2w',b'ZzC-ngWslLIw2hZbWFs3VQ',4,"b""No more discount with the boyd gaming card. ...",0,0,b'2012-05-05',0,0,0,1,0
2,b'review',1,b'2Ka-a7ayZy6fcxqdTxmyhA',b'dOud83AcbDgX5SwhNzVJPw',b'K2-KRkqecWxmwqPHQWVZbA',4,"b""Well, it is a gas station, with lots of pump...",1,0,b'2013-04-19',0,0,0,1,0
3,b'review',0,b'2Ka-a7ayZy6fcxqdTxmyhA',b'FFViSKnujFIkjm60P7li3Q',b'K8j6tfvUOw2QjThjx8mlqA',1,"b""Are you kidding. Went here at night and the...",0,0,b'2016-08-06',1,0,0,0,0
4,b'review',3,b'2Ka-a7ayZy6fcxqdTxmyhA',b'IE3xHWz5aMaeiarwLK68RA',b'JYcCYNWs8Ul6ewG5kCYW4Q',1,"b""I hate this gas station. Unfortunately it is...",3,5,b'2014-06-02',1,0,0,0,0


In [18]:
checkins.columns = ['bizID','type','datetime']
checkins.describe()

,bizID,type,datetime
count,50000,50000,50000
unique,50000,1,47614
top,b'_Fy3ALeANfS_ziaB7wOE8g',b'checkin',['Sat-18:1']
freq,1,50000,43


In [19]:
#this is to strip the 'b' tags from bizID, delete if done in SQL before import
checkins['bizID'] = checkins['bizID'].str[2:24]
reviews['bizID'] = reviews['bizID'].str[2:24]
biz['bizID'] = biz['bizID'].str[2:24]


In [25]:
#get list of unique biz and create df
#uniquebiz = []
#for x in reviews['bizID']:
    #if x not in uniquebiz:
        #uniquebiz.append(x)
import numpy as np
uniquebiz=np.unique(reviews['bizID'])
bizrevs = pd.DataFrame(uniquebiz)
bizrevs.columns = ['bizID']
bizrevs = pd.merge(bizrevs,biz[['bizID','name','address']],on='bizID', how='left')

**Note: merge in cell above won't result in matches for name/address because of 50 row limit put in place. should work after removing limit**

In [ ]:
### If possible, we should try to use predefined pandas/numpy packages to do the aggregations below. 
### Looping is very memory intensive and is hanging up with 50K records.

#declare rest of columns
bizrevs['reviews'] = ""
bizrevs['checkins'] = 0
bizrevs['stars_1'] = 0
bizrevs['stars_2'] = 0
bizrevs['stars_3'] = 0
bizrevs['stars_4'] = 0
bizrevs['stars_5'] = 0
bizrevs['reviews_whole_words'] = ""

#look up each unique biz ID in reviews table
for i in range(len(bizrevs['bizID'])):
    #add text of review to reviews column if biz IDs match
    for x in range(len(reviews['bizID'])):
        if bizrevs['bizID'][i] == reviews['bizID'][x]:
            bizrevs['reviews'][i] = bizrevs['reviews'][i] + reviews['text'][x]
    #count number of checkins
    for y in range(len(checkins['bizID'])):
        if bizrevs['bizID'][i] == checkins['bizID'][y]:
            bizrevs['checkins'][i] = bizrevs['checkins'][i] + 1
    #count number of reviews with each star rating
    for z in range(len(reviews['bizID'])):
        if bizrevs['bizID'][i] == reviews['bizID'][z]:
            bizrevs['stars_1'][i] = bizrevs['stars_1'][i] + reviews['stars_1'][z]
            bizrevs['stars_2'][i] = bizrevs['stars_2'][i] + reviews['stars_2'][z]
            bizrevs['stars_3'][i] = bizrevs['stars_3'][i] + reviews['stars_3'][z]
            bizrevs['stars_4'][i] = bizrevs['stars_4'][i] + reviews['stars_4'][z]
            bizrevs['stars_5'][i] = bizrevs['stars_5'][i] + reviews['stars_5'][z]
    #extract whole words from reviews
    bizrevs['reviews_whole_words'][i] = ' '.join(re.findall('[A-Za-z]+', bizrevs['reviews'][i]))
    
bizrevs.head()

/Users/cmvoelker/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/Users/cmvoelker/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/cmvoelker/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/cmvoelker/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:26: SettingWit

In [ ]:
#import toronto inspection data
cur.execute("SELECT * FROM public.trnt_inspn_agg LIMIT 50")
insp = pd.DataFrame(cur.fetchall())
insp.columns = ['insp_bizID','insp_biz_name','insp_biz_address','insp_date','insp_count_minor','insp_count_significant','insp_count_crucial','insp_count_na','insp_total_count_cs']
insp.head()

In [ ]:
#declare function to calculate levenshtein distance between 2 strings (not case sensitive)
def lev(string1, string2):
    #delete the ".lower()" in the following to lines to make distance case sensitive
    s1=string1.lower()
    s2=string2.lower()
    m=len(s1)+1
    n=len(s2)+1

    tbl = {}
    for i in range(m): tbl[i,0]=i
    for j in range(n): tbl[0,j]=j
    for i in range(1, m):
        for j in range(1, n):
            cost = 0 if s1[i-1] == s2[j-1] else 1
            tbl[i,j] = min(tbl[i, j-1]+1, tbl[i-1, j]+1, tbl[i-1, j-1]+cost)

    return tbl[i,j]

#test the function
print(lev('Hello',"hello"))
print(lev('dock','duck'))
print(lev('st','saint'))

**note: next cell should return matches once we include more than the 50 rows (fingers crossed)**

In [ ]:
#set value of levenshtein distance threshold (4 means only distances of 3 and lower would be considered)
lev_dist_threshold = 4

#loop through each yelp bizID and find the restaurant with closest lev distance (currently matches using name only)
bizrevs['lev_dist'] = lev_dist_threshold
bizrevs['insp_bizID'] = ""
for i in range(len(bizrevs['bizID'])):
    for x in range(len(insp['insp_bizID'])):
        dist = lev(str(bizrevs['name'][i]),str(insp['insp_biz_name'][x]))
        if dist < bizrevs['lev_dist'][i]:
            bizrevs['lev_dist'][i] = dist
            bizrevs['insp_bizID'][i] = insp['insp_bizID'][x]

df = pd.merge(bizrevs,insp,on='insp_bizID', how='left')            

df.head()